In [1]:
from collections import defaultdict
from datetime import datetime
import json
import os
import sys
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import numpy as np
import pandas as pd
from tqdm import tqdm

sys.path.append(os.path.abspath(".."))

from config import (
    RAW_DATA_PATH,
    FILTERED_DATA_PATH,
    TRAIN_DATA_PATH,
    VAL_DATA_PATH,
    TEST_DATA_PATH,
    METADATA_PATH,
    EMBEDDINGS_TITLE_PATH,
    EMBEDDINGS_DESCR_PATH,
    EPISODE_DESCRIPTION_PATH,
    RANDOM_STATE,
    N_RECOMMENDATIONS,
    N_EPOCHS,
    RECOMMENDATIONS_PATH,
    TRANSFORMED_DATA_PATH,
    SPLIT_DATE_TRAIN_VAL,
    SPLIT_DATE_VAL_TEST,
)
import utils.utils as utils

In [2]:
# importing raw data
raw_df = pd.read_parquet("../" + RAW_DATA_PATH)

# importing training data
train_df = pd.read_parquet("../" + TRAIN_DATA_PATH)

# importing validation data
val_df = pd.read_parquet("../" + VAL_DATA_PATH)

# importing test data
test_df = pd.read_parquet("../" + TEST_DATA_PATH)

# importing metadata
meta_df = pd.read_parquet("../" + METADATA_PATH)

# importing episode descriptions
descr_df = pd.read_parquet("../" + EPISODE_DESCRIPTION_PATH)

# importing the transformed data
transformed_df = pd.read_parquet("../" + TRANSFORMED_DATA_PATH)

# importing the filtered data
filtered_df = pd.read_parquet("../" + FILTERED_DATA_PATH)

# importing title embeddings
title_embeddings = pd.read_parquet("../" + EMBEDDINGS_TITLE_PATH)

# importing description embeddings
descr_embeddings = pd.read_parquet("../" + EMBEDDINGS_DESCR_PATH)

In [37]:
train_w_meta = train_df.merge(meta_df, how="left", on="prd_number")

In [3]:
meta_df

,prd_number,series_title,unique_title,pub_date,episode_duration,genre,branding_channel,mother_channel,category,episode_description,episode_title
0,11031452026,Radiofortællinger,Radiofortællinger: Lykkelige ulykker_11031452026,2020-03-21,1604,-,DR P1,-,Oplysning og kultur,Hvordan er det at se hele ens professionelle l...,Lykkelige ulykker
1,11031452036,Radiofortællinger,Radiofortællinger: Kunsten at vælge et andet m...,2018-08-21,1621,-,DR P1,-,Oplysning og kultur,Alle mennesker må på et eller andet tidspunkt ...,Kunsten at vælge et andet menneske fra
2,11031452096,Radiofortællinger,Radiofortællinger: Ved grænsen til stalking_11...,2024-11-20,1623,Fakta og debat,DR P1,DR P1,Oplysning og kultur,Sandra Virginia Bereza har været tæt på grænse...,Ved grænsen til stalking
3,11031452116,Radiofortællinger,Radiofortællinger: Ind og ud af troen_11031452116,2024-09-30,1626,Fakta og debat,DR P1,DR P1,Oplysning og kultur,"På trods af sin unge alder, 19 år, har Asta al...",Ind og ud af troen
4,11031452216,Radiofortællinger,Radiofortællinger: Flyskræk_11031452216,2024-06-06,1626,Fakta og debat,DR P1,DR P1,Oplysning og kultur,Er du bange for at flyve? Du er ikke den enest...,Flyskræk
...,...,...,...,...,...,...,...,...,...,...,...
21051,19388840409,-,Krop og bevægelse_19388840409,2015-09-09,569,None,None,None,Præsentation og services,"- med Inge Gotved,\nMargit Riis-Vestergaard og...",Krop og bevægelse
21052,19388840410,-,Krop og bevægelse_19388840410,2018-10-13,563,None,None,None,Præsentation og services,"- med Inge Gotved,\nMargit Riis-Vestergaard og...",Krop og bevægelse
21053,19388840411,-,Krop og bevægelse_19388840411,2018-09-30,571,None,None,None,Præsentation og services,"- med Inge Gotved,\nMargit Riis-Vestergaard og...",Krop og bevægelse
21054,19388840412,-,Krop og bevægelse_19388840412,2018-11-16,566,None,None,None,Præsentation og services,"- med Inge Gotved,\nMargit Riis-Vestergaard og...",Krop og bevægelse


In [5]:
meta_df[meta_df["episode_description"].str.contains("Frederiksen", case=False, na=False)]

,prd_number,series_title,unique_title,pub_date,episode_duration,genre,branding_channel,mother_channel,category,episode_description,episode_title
178,11031902483,Radiofortællinger,I en danskers blodspor 1:7 - Hvad bruger man e...,2019-10-31,1555,-,DR P1,-,Oplysning og kultur,Tækkemanden og våbenhandleren Peter Frederikse...,I en danskers blodspor 1:7 - Hvad bruger man e...
179,11031902493,Radiofortællinger,I en danskers blodspor 2:7 - Begrundet mistank...,2019-10-31,1625,-,DR P1,-,Oplysning og kultur,Journalisten Helle Maj og fotografen Jørn Stje...,I en danskers blodspor 2:7 - Begrundet mistank...
180,11031902503,Radiofortællinger,I en danskers blodspor 3:7 - Kønsdele som trof...,2019-10-31,1575,-,DR P1,-,Oplysning og kultur,Den danske tækkemand og våbenhandler Peter Fre...,I en danskers blodspor 3:7 - Kønsdele som trof...
223,11032007015,Klog på Sprog,Klog på Sprog_11032007015,2020-01-03,3366,-,DR P1,-,Oplysning og kultur,"Statsminister Mette Frederiksen brugte ordet ""...",
238,11032007495,Klog på Sprog,Klog på Sprog: Kancelli-sprog og bureaukratisk...,2020-12-04,3360,-,DR P1,-,Oplysning og kultur,"Får du sommetider breve og beskeder i E-boks, ...",Kancelli-sprog og bureaukratiske formuleringer
...,...,...,...,...,...,...,...,...,...,...,...
18721,16122393113,Op af kaninhullet,Op af kaninhullet: Astrologi - Styrer stjerner...,2023-04-25,2740,Fakta og debat,dr.dk radio,-,Oplysning og kultur,Styrer du selv din skæbne eller er det i virke...,Astrologi - Styrer stjernerne dit liv?
18725,16122393117,Op af kaninhullet,Op af kaninhullet: Operation Gladio 2:2_161223...,2023-05-23,1926,Fakta og debat,dr.dk radio,-,Oplysning og kultur,"Det var simpelthen for stort et kaninhul til, ...",Operation Gladio 2:2
18730,16122393122,Op af kaninhullet,Op af kaninhullet: Lytternes kaninhuller #2_16...,2023-06-27,2528,Fakta og debat,dr.dk radio,-,Oplysning og kultur,"Vi er nået til det sidste afsnit ""Op af kaninh...",Lytternes kaninhuller #2
19030,16122413252,Guld og grønne skove,Guld og grønne skove: Åbent hus_16122413252,2024-06-18,3257,Fakta og debat,-,DR P1,Oplysning og kultur,Mens landmændene jamrer over udsigten til en C...,Åbent hus


In [6]:
# downloading stopwords
nltk.download('stopwords')

# dictionaries to hold embeddings
embedding_dicts = defaultdict(dict)

# Danish stopwords
stop_words = set(stopwords.words('danish'))

# Danish stemmer
stemmer = SnowballStemmer("danish")

text = 'Statsminister Mette Frederiksen brugte ordet "Børn" mere end 20 gange i sin nytårstale til nationen.'

# tokenizing the text 
words = text.split()

# removing stopwords
filtered_words = [word for word in words if word.lower() not in stop_words]

# joining the words back into a sentence
filtered_text = " ".join(filtered_words)
print(filtered_text)
# stemming the filtered words
stemmed_words = [stemmer.stem(word) for word in filtered_words]

# joining the words back into a sentence
stemmed_text = " ".join(stemmed_words)
print(stemmed_text)

Statsminister Mette Frederiksen brugte ordet "Børn" mere 20 gange nytårstale nationen.
statsminist met frederiks brugt ord "børn" mer 20 gang nytårstal nationen.


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sewr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
